# Overview

This tutorial shows you how the library interacts with custom mods. To do so we create a generic mod that runs code on the vehicle and game engine side.

In this tutorial we differentiate between mod and lua module:

mod - refers to a collection of files that change the behavior of the game (see the zip file)

lua module - refers to a lua 'library' that has a global name containing a table 


## Setup

These are the usual imports to create a simple scenario. As the point is to show the mod support, nothing is happening in the scenario. The mod only prints some text to the console, so take a look at the console or the log file you can find in "~\Documents\BeamNG.research". Note that the console only flushes the last line to the log file if another console output line is generated.



In [1]:
import os
import zipfile
from pathlib import Path

from beamngpy import BeamNGpy, Scenario, Vehicle, setup_logging

In [2]:
setup_logging()
beamng = BeamNGpy('localhost', 64256)
scenario = Scenario('smallgrid', 
                    "On how to use custom mods")
vehicle = Vehicle('ego_vehicle', model='etk800', licence='AI', extensions=["vehicleEngineCode"])
scenario.add_vehicle(vehicle, pos=(0,0,0), rot_quat=(0, 0, 1, 0))
scenario.make(beamng)

2020-10-19 15:24:10,682 INFO     Started BeamNGpy logging.
2020-10-19 15:24:10,685 DEBUG    Determined BeamNG.* binary to be: C:\Users\Smeik\BeamNG\game\Bin64\BeamNG.drive.x64.exe


## Creating a Simple Mod

Mods can be installed by creating a zip directory in the ".../Documents/BeamNG.reasearch/mods/" folder. They have to recreate the exact same file structure as you find in the game directory in "BeamNG.research.vX.X.X.X/lua".

What happens here is that the two lua files in the BeamNGpy directory are zipped into a mod.

In [3]:
# setting up mod
myModPath = beamng.user / 'mods' / 'genericResearchMod.zip' 
geCode = 'gameEngineCode.lua'
zipGEpath = str(Path('lua') / 'ge' / 'extensions' / 'util' / geCode)
veCode = 'vehicleEngineCode.lua'
zipVEpath = str(Path('lua') / 'vehicle' / 'extensions' / veCode)
localDir = Path(os.path.abspath('.'))
with zipfile.ZipFile(str(myModPath), 'w') as ziph:
    ziph.write(localDir / geCode, arcname=zipGEpath)
    ziph.write(localDir / veCode, arcname=zipVEpath)

2020-10-19 15:24:11,640 WARNING  c:\Users\Smeik\BeamNGpy\bng\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



## Testing the mod

To test the mod we start BeamNG.research and give the python BeamNG class the location of the gameengine mod within the "genericResearchMod.zip/lua/ge/extensions/" directory. This is necessary to register the file as its own lua module within the game.

After registration, it is available as util_gameEngineCode within the game - try typing `dump(util_gameEngineCode)` in the game's command prompt

In [4]:
bng = beamng.open(extensions=["util/gameEngineCode"])
bng.load_scenario(scenario)
bng.start_scenario()

2020-10-19 15:24:13,594 INFO     Opening BeamNPy instance...
2020-10-19 15:24:13,606 INFO     Started BeamNGpy server on localhost:64256
2020-10-19 15:24:13,608 DEBUG    Starting BeamNG process...
2020-10-19 15:24:24,713 DEBUG    Connection established. Awaiting "hello"...
2020-10-19 15:24:24,715 INFO     Started BeamNGpy communicating on ('127.0.0.1', 61335)
2020-10-19 15:24:36,053 DEBUG    Starting vehicle server for V:ego_vehicle on: localhost:64257


## How to Call Lua Module Functions

BeamNG.research and BeamNGpy communicate via sockets. 
Function parameters are send with the help of python dicitonaries that are available as lua tables in the game. 
The mini mod from this tutorial follows the convention the BeamNGpy library established: Every value for the 'type' key helps identifying the appropriate handler, which for 'Foo' is expected to be 'handleFoo'.
The checkmessage function in researchGE.lua and researchVE.lua checks whether a corresponding function is locally available and, if not, calls the 'onSocketMessage' hook for every extension. 

In [5]:
# executing 'handleFoo' from gameEngineCode
data = dict(type='Foo', someName = 'YourName')
bng.send(data)
# executing 'handleBar' from vehicleEngineCode
data = dict(type='Bar', text = 'lorem ipsum...' )
vehicle.send(data)

### Can't see anything special?

If you have trouble finding the messages in the log file or command prompt, search for 'gameEngineCode' or 'vehicleEngineCode'. These are the respective log tags of the modules.

In [6]:
bng.close()

2020-10-19 15:25:44,637 INFO     Closing BeamNGpy instance...
2020-10-19 15:25:44,638 DEBUG    Killing BeamNG process...
2020-10-19 15:25:45,022 WARNING  C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.1776.0_x64__qbz5n2kfra8p0\lib\subprocess.py:942: ResourceWarning: subprocess 14448 is still running
  _warn("subprocess %s is still running" % self.pid,

